In [17]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.optim as optim
import os
from sklearn.metrics import mean_squared_error
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
torch.manual_seed(1)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [18]:
train_data = pd.read_csv("../input/coffeecsmr/train_ (1).csv")
test_data = pd.read_csv("../input/coffeecsmr/test_ (1).csv")
y_data = pd.read_csv("../input/coffeecsmr/y.csv")
submit = pd.read_csv("../input/coffeecsmr/sample_submission (1).csv")
train_data

,gov_dn_cd,bntr_nm,signgu_nm,adstrd_nm,de,sex_se,year_se
0,4146352000,경기도,용인시 기흥구,구갈동,20201025,F,35
1,1156065000,서울특별시,영등포구,신길3동,20201015,F,20
2,4139064000,경기도,시흥시,능곡동,20200812,M,50
3,1114058000,서울특별시,중구,장충동,20200923,F,30
4,4121056000,경기도,광명시,광명5동,20200923,F,30
...,...,...,...,...,...,...,...
576136,4713057000,경상북도,경주시,황남동,20201020,M,35
576137,1159068000,서울특별시,동작구,신대방2동,20200912,M,25
576138,4145057000,경기도,하남시,풍산동,20201027,M,20
576139,4615061000,전라남도,순천시,장천동,20201023,F,45


In [19]:
x_train = train_data.drop(['gov_dn_cd'], axis=1)
y_train = y_data['cnsmr_popltn_co']
x_test = test_data.drop(['gov_dn_cd'], axis=1)

In [20]:
from sklearn import preprocessing

for col in x_train.columns:
    le = preprocessing.LabelEncoder()
    le.fit(pd.concat([x_train[col],x_test[col]],axis=0))
    x_train[col] = le.transform(x_train[col])
    x_test[col] = le.transform(x_test[col])
    
# le = preprocessing.LabelEncoder()
# le.fit(pd.concat([x_train.adstrd_nm,x_test.adstrd_nm],axis=0))
# x_train.adstrd_nm = le.transform(x_train.adstrd_nm)
# x_test.adstrd_nm = le.transform(x_test.adstrd_nm)

# le2 = preprocessing.LabelEncoder()
# le2.fit(pd.concat([x_train.bntr_nm,x_test.bntr_nm],axis=0))
# x_train.bntr_nm = le2.transform(x_train.bntr_nm)
# x_test.bntr_nm = le2.transform(x_test.bntr_nm)

# le3 = preprocessing.LabelEncoder()
# le3.fit(pd.concat([x_train.signgu_nm,x_test.signgu_nm],axis=0))
# x_train.signgu_nm = le3.transform(x_train.signgu_nm)
# x_test.signgu_nm = le3.transform(x_test.signgu_nm)

# le4 = preprocessing.LabelEncoder()
# le4.fit(pd.concat([x_train.de,x_test.de],axis=0))
# x_train.de = le4.transform(x_train.de)
# x_test.de = le4.transform(x_test.de)

# le5 = preprocessing.LabelEncoder()
# le5.fit(pd.concat([x_train.sex_se,x_test.sex_se],axis=0))
# x_train.sex_se = le5.transform(x_train.sex_se)
# x_test.sex_se = le5.transform(x_test.sex_se)

# le6 = preprocessing.LabelEncoder()
# le6.fit(pd.concat([x_train.year_se,x_test.year_se],axis=0))
# x_train.year_se = le6.transform(x_train.year_se)
# x_test.year_se = le6.transform(x_test.year_se)

ad = pd.concat([x_train.adstrd_nm,x_test.adstrd_nm],axis=0).max()+1
bn = pd.concat([x_train.bntr_nm,x_test.bntr_nm],axis=0).max()+1
si = pd.concat([x_train.signgu_nm,x_test.signgu_nm],axis=0).max()+1
de = pd.concat([x_train.de,x_test.de],axis=0).max()+1
sex = pd.concat([x_train.sex_se,x_test.sex_se],axis=0).max()+1
year = pd.concat([x_train.year_se,x_test.year_se],axis=0).max()+1

In [21]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576141 entries, 0 to 576140
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   bntr_nm    576141 non-null  int64
 1   signgu_nm  576141 non-null  int64
 2   adstrd_nm  576141 non-null  int64
 3   de         576141 non-null  int64
 4   sex_se     576141 non-null  int64
 5   year_se    576141 non-null  int64
dtypes: int64(6)
memory usage: 26.4 MB


In [22]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 283771 entries, 0 to 283770
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   bntr_nm    283771 non-null  int64
 1   signgu_nm  283771 non-null  int64
 2   adstrd_nm  283771 non-null  int64
 3   de         283771 non-null  int64
 4   sex_se     283771 non-null  int64
 5   year_se    283771 non-null  int64
dtypes: int64(6)
memory usage: 13.0 MB


In [23]:
x_train

,bntr_nm,signgu_nm,adstrd_nm,de,sex_se,year_se
0,1,150,209,85,0,5
1,8,136,1303,75,0,2
2,1,112,394,11,1,8
3,8,178,1892,53,0,4
4,1,25,172,53,0,4
...,...,...,...,...,...,...
576136,3,11,2366,80,1,5
576137,8,65,1314,42,1,3
576138,1,213,2222,87,1,2
576139,11,111,1891,83,0,7


In [24]:
x_train = torch.LongTensor(np.array(x_train)).to(device)
y_train = torch.FloatTensor(np.array(y_train)).to(device).reshape(-1,1)
x_test = torch.LongTensor(np.array(x_test)).to(device)
__ = torch.FloatTensor(np.array(submit['cnsmr_popltn_co'])).to(device).reshape(-1,1)

In [25]:
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True) # 원래 배치 1024

test_dataset = TensorDataset(x_test, __)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False) # 원래 배치 1024

In [26]:
from tqdm.notebook import tqdm

class DNN(torch.nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        K = 150
        self.AD = torch.nn.Embedding(num_embeddings=ad, embedding_dim=K)
        self.BN = torch.nn.Embedding(num_embeddings=bn, embedding_dim=K)
        self.SI = torch.nn.Embedding(num_embeddings=si, embedding_dim=K)
        self.DE = torch.nn.Embedding(num_embeddings=de, embedding_dim=K)
        self.SEX = torch.nn.Embedding(num_embeddings=sex, embedding_dim=K)
        self.YEAR = torch.nn.Embedding(num_embeddings=year, embedding_dim=K)

        
        self.relu = torch.nn.ReLU()

        self.Linear1 = torch.nn.Linear(900, 1024)
        self.Linear2 = torch.nn.Linear(1024, 2048)
        self.Linear3 = torch.nn.Linear(2048, 4096)
        self.Linear4 = torch.nn.Linear(4096, 2048)
        self.Linear5 = torch.nn.Linear(2048, 1024)
        self.Linear6 = torch.nn.Linear(1024, 512)
        self.Linear7 = torch.nn.Linear(512, 1)
        
        self.bn1 = torch.nn.BatchNorm1d(1024)
        self.bn2 = torch.nn.BatchNorm1d(2048)
        self.bn3 = torch.nn.BatchNorm1d(4096)
        self.bn4 = torch.nn.BatchNorm1d(2048)
        self.bn5 = torch.nn.BatchNorm1d(1024)
        self.bn6 = torch.nn.BatchNorm1d(512)
        
        self.model = torch.nn.Sequential(self.Linear1, self.bn1 ,self.relu,# self.drop,
                                         self.Linear2, self.bn2 ,self.relu,# self.drop,
                                         self.Linear3, self.bn3 ,self.relu,# self.drop,
                                         self.Linear4, self.bn4 ,self.relu,# self.drop,
                                         self.Linear5, self.bn5 ,self.relu,# self.drop,
                                         self.Linear6, self.bn6 ,self.relu,# self.drop,
                                         self.Linear7)
        self.weight_init()
    def weight_init(self):
    # model에 있는 모듈을 하나씩 불러와 Linear 모듈이면 xavier로 weight 초기화
        for module in self.model:
            if isinstance(module, torch.nn.Linear):
                torch.nn.init.xavier_uniform_(module.weight)
                module.bias.data.fill_(0.01)

    def forward(self, x):
    # Tensor 입력을 받아서 forwarding 진행
        BN = torch.flatten(self.BN(x[:,:1]),1)
        SI = torch.flatten(self.SI(x[:,1:2]),1)
        AD = torch.flatten(self.AD(x[:,2:3]),1)
        DE = torch.flatten(self.DE(x[:,3:4]),1)
        SEX = torch.flatten(self.SEX(x[:,4:5]),1)
        YEAR = torch.flatten(self.YEAR(x[:,5:6]),1)
        
        R = torch.cat([AD, BN, SI, DE, SEX, YEAR],1)
        
        return self.model(R)
        

model = DNN().to(device)

epochs = 48
lr = 1e-3
batch_size = 512
optimizer = optim.Adam(model.parameters(), lr)
loss_fn = torch.nn.MSELoss().to(device)


total_batch = len(train_loader)
model.train()

for epoch in range(epochs + 1):
    avg_loss = 0
    
    for X, Y in tqdm(train_loader):
        
        hypothesis = model(X)
        loss = loss_fn(hypothesis, Y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        avg_loss += loss / total_batch
    
    print("Epoch : {:4d} Loss : {:.7f}".format(epoch, avg_loss**0.5))

  0%|          | 0/1126 [00:00<?, ?it/s]

Epoch :    0 Loss : 338.3526611


  0%|          | 0/1126 [00:00<?, ?it/s]

Epoch :    1 Loss : 232.9368591


  0%|          | 0/1126 [00:00<?, ?it/s]

...

Epoch :   46 Loss : 43.7044411


  0%|          | 0/1126 [00:00<?, ?it/s]

Epoch :   47 Loss : 46.1334114


  0%|          | 0/1126 [00:00<?, ?it/s]

Epoch :   48 Loss : 38.9785004


In [27]:
# 45.5341873  test 57.3789997244

# 41.9021759  test 56.3079150638 

# 42.7822227 test 54.6784278189

# 1e-4 70epoch simillar to 42.782227

In [28]:
submit = pd.read_csv("../input/coffeecsmr/sample_submission (1).csv")
submit

,index,cnsmr_popltn_co
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
283766,283766,0
283767,283767,0
283768,283768,0
283769,283769,0


In [29]:
model.eval()
test_predict = torch.Tensor()
with torch.no_grad():
    for x_, _ in tqdm(test_loader):
        predict1 = model(x_)
        test_predict = torch.cat([test_predict.cpu(), predict1.cpu()])
test_predict

  0%|          | 0/555 [00:00<?, ?it/s]

tensor([[ 31.4056],
        [624.5490],
        [ 43.1120],
        ...,
        [ 34.4854],
        [ 97.9572],
        [155.8929]])

In [30]:
submit['cnsmr_popltn_co'] = test_predict.cpu()

In [31]:
submit.to_csv("submit.csv",index=False)